<p align = 'left'>
<img src = 'https://i.imgur.com/8LTNXxF.jpg'>
</p>

---
## **Data Scientist & Analyst**

*by [Paulo Henrique de Melo Ferreira](https://www.linkedin.com/in/paulo-henrique-ferreira-20151374/)*

---

<p style="text-align: justify;">
O atual projeto visa analisar dados da <b>Alura Voz</b>, a qual consiste em uma empresa de telecomunicações. O objetivo desta análise é obter insights acerca do <b>Churn Rate</b> da empresa e encontrar fatores que estejam influenciando essa métrica.
Os dados desta análise foram obtidos através de uma <b>API</b> por webscrapping.

Iniciaremos este Projeto pelo carregamento dos pacotes necessários para análise de dados.

In [83]:
# Carregando pacostes para análise de dados
import pandas as pd
import numpy as np

In [84]:
# Aqui carregamos os dados por meio de um arquivo no formato JSON, disponibilizado pelo github
dados = pd.read_json('https://raw.githubusercontent.com/sthemonica/alura-voz/main/Dados/Telco-Customer-Churn.json')
dados.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [85]:
# Como o carregamento para o Data Frame não ocorreu como o esperado, vamos explorar o que cada campo armazena em sua primeira linha.
dados.customer[0]
# Aqui observamos que são dados gerais referentes ao cliente

{'gender': 'Female',
 'SeniorCitizen': 0,
 'Partner': 'Yes',
 'Dependents': 'Yes',
 'tenure': 9}

In [86]:
# Dados acerca da linha do cliente
dados.phone[0]

{'PhoneService': 'Yes', 'MultipleLines': 'No'}

In [87]:
# Dados a respeito de serviços de internet
dados.internet[0]

{'InternetService': 'DSL',
 'OnlineSecurity': 'No',
 'OnlineBackup': 'Yes',
 'DeviceProtection': 'No',
 'TechSupport': 'Yes',
 'StreamingTV': 'Yes',
 'StreamingMovies': 'No'}

In [88]:
# Dados acerca do contrato e formas de pagamento
dados.account[0]

{'Contract': 'One year',
 'PaperlessBilling': 'Yes',
 'PaymentMethod': 'Mailed check',
 'Charges': {'Monthly': 65.6, 'Total': '593.3'}}

In [89]:
# A partir daqui vamos normalizar os dados, a fim de poder trabalhar melhor com o Data Frame
customer_dados = pd.json_normalize(dados.customer)

In [90]:
phone_dados = pd.json_normalize(dados.phone)
phone_dados.head()

,PhoneService,MultipleLines
0,Yes,No
1,Yes,Yes
2,Yes,No
3,Yes,No
4,Yes,No


In [91]:
internet_dados = pd.json_normalize(dados.internet)
internet_dados.head()

,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,DSL,No,Yes,No,Yes,Yes,No
1,DSL,No,No,No,No,No,Yes
2,Fiber optic,No,No,Yes,No,No,No
3,Fiber optic,No,Yes,Yes,No,Yes,Yes
4,Fiber optic,No,No,No,Yes,Yes,No


In [92]:
account_dados = pd.json_normalize(dados.account)
account_dados.head()

,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,One year,Yes,Mailed check,65.6,593.3
1,Month-to-month,No,Mailed check,59.9,542.4
2,Month-to-month,Yes,Electronic check,73.9,280.85
3,Month-to-month,Yes,Electronic check,98.0,1237.85
4,Month-to-month,Yes,Mailed check,83.9,267.4


In [93]:
# Definindo uma função para concatenar todas as tabelas ao longo das colunas
def normalize_json(dataframe):
    return_dataframe = pd.DataFrame()
    for column in list(dados.columns[2:]):
        dataframe_column = pd.json_normalize(dataframe[column])
        return_dataframe = pd.concat([return_dataframe, dataframe_column], axis=1)

    return pd.concat([dataframe[list(dados.columns[:2])], return_dataframe], axis=1)

In [94]:
# Aplicando a função e rearmazenando os dados na respectiva variável, vamos também visualizar o resultado
dados = normalize_json(dados)
dados

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.9
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


Agora com os dados devidamente armazenados em um Data Frame, podemos iniciar a limpeza de dados deste. Mas antes, vamos entender do que se tratam cada uma das variáveis de análise.

In [95]:
# Consultando os nomes dos das colunas no Data Frame
dados.columns.values

array(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'Charges.Monthly', 'Charges.Total'], dtype=object)

Para facilitar o entendimento das colunas, vamos traduzi-las para conferir maior acessibilidade à equipe de trabalho.

In [96]:
nomes = {'customerID': 'ID', 'Churn': 'Evasao', 'gender': 'Genero', 'SeniorCitizen': 'Senior', 'Partner': 'Casado(a)',
       'Dependents': 'Dependentes', 'tenure': 'Posse', 'PhoneService': 'ServicoTelefonico', 'MultipleLines': 'MultiplasLinhas',
       'InternetService': 'ServicodeInternet', 'OnlineSecurity': 'SegurancaOnline', 'OnlineBackup': 'BackupOnline',
       'DeviceProtection': 'ProtecaodeDispositivo', 'TechSupport': 'SuporteTecnico', 'StreamingTV': 'StremingTV',
       'StreamingMovies': 'StremingFilmes', 'Contract': 'Contrato', 'PaperlessBilling': 'CobrancasemPapel', 'PaymentMethod': 'MetododePagamento',
       'Charges.Monthly': 'CobrancasMensais', 'Charges.Total': 'CobrancasTotais'}

In [97]:
dados.rename(columns = nomes, inplace = True)
dados

,ID,Evasao,Genero,Senior,Casado(a),Dependentes,Posse,ServicoTelefonico,MultiplasLinhas,ServicodeInternet,...,BackupOnline,ProtecaodeDispositivo,SuporteTecnico,StremingTV,StremingFilmes,Contrato,CobrancasemPapel,MetododePagamento,CobrancasMensais,CobrancasTotais
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.9
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


In [98]:
# Verificando valores únicos das colunas
dados['MultiplasLinhas'].unique()

array(['No', 'Yes', 'No phone service'], dtype=object)

In [99]:
dados['ServicodeInternet'].unique()

array(['DSL', 'Fiber optic', 'No'], dtype=object)

In [100]:
dados['Contrato'].unique()

array(['One year', 'Month-to-month', 'Two year'], dtype=object)

In [101]:
dados['MetododePagamento'].unique()

array(['Mailed check', 'Electronic check', 'Credit card (automatic)',
       'Bank transfer (automatic)'], dtype=object)

In [102]:
dados['Senior'].unique()

array([0, 1])

In [103]:
# Traduzindo valores contidos nas colunas
dados.replace({'Yes': 'Sim', 'No': 'Nao', 'Female': 'Mulher', 'Male': 'Homem', 'No phone service': 'Sem servico telefonico', 'Fiber optic': 'Fibra optica', 'One year': 'Anual', 'Month-to-month': 'Mensal', 'Two year': 'Dois anos', 'Mailed check': 'Correspondencia', 'Electronic check': 'Eletronico', 'Credit card (automatic)': 'Cartao de credito', 'Bank transfer (automatic)': 'Debito automatico'}, inplace = True)
dados

,ID,Evasao,Genero,Senior,Casado(a),Dependentes,Posse,ServicoTelefonico,MultiplasLinhas,ServicodeInternet,...,BackupOnline,ProtecaodeDispositivo,SuporteTecnico,StremingTV,StremingFilmes,Contrato,CobrancasemPapel,MetododePagamento,CobrancasMensais,CobrancasTotais
0,0002-ORFBO,Nao,Mulher,0,Sim,Sim,9,Sim,Nao,DSL,...,Sim,Nao,Sim,Sim,Nao,Anual,Sim,Correspondencia,65.60,593.3
1,0003-MKNFE,Nao,Homem,0,Nao,Nao,9,Sim,Sim,DSL,...,Nao,Nao,Nao,Nao,Sim,Mensal,Nao,Correspondencia,59.90,542.4
2,0004-TLHLJ,Sim,Homem,0,Nao,Nao,4,Sim,Nao,Fibra optica,...,Nao,Sim,Nao,Nao,Nao,Mensal,Sim,Eletronico,73.90,280.85
3,0011-IGKFF,Sim,Homem,1,Sim,Nao,13,Sim,Nao,Fibra optica,...,Sim,Sim,Nao,Sim,Sim,Mensal,Sim,Eletronico,98.00,1237.85
4,0013-EXCHZ,Sim,Mulher,1,Sim,Nao,3,Sim,Nao,Fibra optica,...,Nao,Nao,Sim,Sim,Nao,Mensal,Sim,Correspondencia,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,Nao,Mulher,0,Nao,Nao,13,Sim,Nao,DSL,...,Nao,Nao,Sim,Nao,Nao,Anual,Nao,Correspondencia,55.15,742.9
7263,9992-RRAMN,Sim,Homem,0,Sim,Nao,22,Sim,Sim,Fibra optica,...,Nao,Nao,Nao,Nao,Sim,Mensal,Sim,Eletronico,85.10,1873.7
7264,9992-UJOEL,Nao,Homem,0,Nao,Nao,2,Sim,Nao,DSL,...,Sim,Nao,Nao,Nao,Nao,Mensal,Sim,Correspondencia,50.30,92.75
7265,9993-LHIEB,Nao,Homem,0,Sim,Sim,67,Sim,Nao,DSL,...,Nao,Sim,Sim,Nao,Sim,Dois anos,Nao,Correspondencia,67.85,4627.65


Após esses processos vamos dar início à limpeza dos dados do nosso Data Frame.

In [104]:
# Compreendendo a composição do Data Frame
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     7267 non-null   object 
 1   Evasao                 7267 non-null   object 
 2   Genero                 7267 non-null   object 
 3   Senior                 7267 non-null   int64  
 4   Casado(a)              7267 non-null   object 
 5   Dependentes            7267 non-null   object 
 6   Posse                  7267 non-null   int64  
 7   ServicoTelefonico      7267 non-null   object 
 8   MultiplasLinhas        7267 non-null   object 
 9   ServicodeInternet      7267 non-null   object 
 10  SegurancaOnline        7267 non-null   object 
 11  BackupOnline           7267 non-null   object 
 12  ProtecaodeDispositivo  7267 non-null   object 
 13  SuporteTecnico         7267 non-null   object 
 14  StremingTV             7267 non-null   object 
 15  Stre

In [105]:
# Verficando os valores inseridos em cada coluna
for coluna in dados.columns:
  print(dados[coluna].value_counts())

0002-ORFBO    1
6614-VBEGU    1
6637-KYRCV    1
6635-MYYYZ    1
6635-CPNUN    1
             ..
3374-TTZTK    1
3374-PZLXD    1
3374-LXDEV    1
3373-YZZYM    1
9995-HOTOH    1
Name: ID, Length: 7267, dtype: int64
Nao    5174
Sim    1869
        224
Name: Evasao, dtype: int64
Homem     3675
Mulher    3592
Name: Genero, dtype: int64
0    6085
1    1182
Name: Senior, dtype: int64
Nao    3749
Sim    3518
Name: Casado(a), dtype: int64
Nao    5086
Sim    2181
Name: Dependentes, dtype: int64
1     634
72    369
2     246
3     207
4     185
     ... 
38     60
39     59
44     54
36     50
0      11
Name: Posse, Length: 73, dtype: int64
Sim    6560
Nao     707
Name: ServicoTelefonico, dtype: int64
Nao                       3495
Sim                       3065
Sem servico telefonico     707
Name: MultiplasLinhas, dtype: int64
Fibra optica    3198
DSL             2488
Nao             1581
Name: ServicodeInternet, dtype: int64
Nao                    3608
Sim                    2078
No internet se

Verificamos que a coluna Evasão contém 224 valores em branco. Como este é o principal fator que consideraremos para a nossa análise, não há sentido em manter estes 224 valores em nossa análise.

In [108]:
dados['Evasao'].value_counts()

Nao    5174
Sim    1869
        224
Name: Evasao, dtype: int64

In [106]:
# Selecionando os dados de interesse para um novo Data Frame
dados_clean = dados[dados['Evasao'] != '']
dados_clean

,ID,Evasao,Genero,Senior,Casado(a),Dependentes,Posse,ServicoTelefonico,MultiplasLinhas,ServicodeInternet,...,BackupOnline,ProtecaodeDispositivo,SuporteTecnico,StremingTV,StremingFilmes,Contrato,CobrancasemPapel,MetododePagamento,CobrancasMensais,CobrancasTotais
0,0002-ORFBO,Nao,Mulher,0,Sim,Sim,9,Sim,Nao,DSL,...,Sim,Nao,Sim,Sim,Nao,Anual,Sim,Correspondencia,65.60,593.3
1,0003-MKNFE,Nao,Homem,0,Nao,Nao,9,Sim,Sim,DSL,...,Nao,Nao,Nao,Nao,Sim,Mensal,Nao,Correspondencia,59.90,542.4
2,0004-TLHLJ,Sim,Homem,0,Nao,Nao,4,Sim,Nao,Fibra optica,...,Nao,Sim,Nao,Nao,Nao,Mensal,Sim,Eletronico,73.90,280.85
3,0011-IGKFF,Sim,Homem,1,Sim,Nao,13,Sim,Nao,Fibra optica,...,Sim,Sim,Nao,Sim,Sim,Mensal,Sim,Eletronico,98.00,1237.85
4,0013-EXCHZ,Sim,Mulher,1,Sim,Nao,3,Sim,Nao,Fibra optica,...,Nao,Nao,Sim,Sim,Nao,Mensal,Sim,Correspondencia,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,Nao,Mulher,0,Nao,Nao,13,Sim,Nao,DSL,...,Nao,Nao,Sim,Nao,Nao,Anual,Nao,Correspondencia,55.15,742.9
7263,9992-RRAMN,Sim,Homem,0,Sim,Nao,22,Sim,Sim,Fibra optica,...,Nao,Nao,Nao,Nao,Sim,Mensal,Sim,Eletronico,85.10,1873.7
7264,9992-UJOEL,Nao,Homem,0,Nao,Nao,2,Sim,Nao,DSL,...,Sim,Nao,Nao,Nao,Nao,Mensal,Sim,Correspondencia,50.30,92.75
7265,9993-LHIEB,Nao,Homem,0,Sim,Sim,67,Sim,Nao,DSL,...,Nao,Sim,Sim,Nao,Sim,Dois anos,Nao,Correspondencia,67.85,4627.65


In [107]:
# Resetando o Index
dados_clean.reset_index(drop = True, inplace = True)
dados_clean

,ID,Evasao,Genero,Senior,Casado(a),Dependentes,Posse,ServicoTelefonico,MultiplasLinhas,ServicodeInternet,...,BackupOnline,ProtecaodeDispositivo,SuporteTecnico,StremingTV,StremingFilmes,Contrato,CobrancasemPapel,MetododePagamento,CobrancasMensais,CobrancasTotais
0,0002-ORFBO,Nao,Mulher,0,Sim,Sim,9,Sim,Nao,DSL,...,Sim,Nao,Sim,Sim,Nao,Anual,Sim,Correspondencia,65.60,593.3
1,0003-MKNFE,Nao,Homem,0,Nao,Nao,9,Sim,Sim,DSL,...,Nao,Nao,Nao,Nao,Sim,Mensal,Nao,Correspondencia,59.90,542.4
2,0004-TLHLJ,Sim,Homem,0,Nao,Nao,4,Sim,Nao,Fibra optica,...,Nao,Sim,Nao,Nao,Nao,Mensal,Sim,Eletronico,73.90,280.85
3,0011-IGKFF,Sim,Homem,1,Sim,Nao,13,Sim,Nao,Fibra optica,...,Sim,Sim,Nao,Sim,Sim,Mensal,Sim,Eletronico,98.00,1237.85
4,0013-EXCHZ,Sim,Mulher,1,Sim,Nao,3,Sim,Nao,Fibra optica,...,Nao,Nao,Sim,Sim,Nao,Mensal,Sim,Correspondencia,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,Nao,Mulher,0,Nao,Nao,13,Sim,Nao,DSL,...,Nao,Nao,Sim,Nao,Nao,Anual,Nao,Correspondencia,55.15,742.9
7039,9992-RRAMN,Sim,Homem,0,Sim,Nao,22,Sim,Sim,Fibra optica,...,Nao,Nao,Nao,Nao,Sim,Mensal,Sim,Eletronico,85.10,1873.7
7040,9992-UJOEL,Nao,Homem,0,Nao,Nao,2,Sim,Nao,DSL,...,Sim,Nao,Nao,Nao,Nao,Mensal,Sim,Correspondencia,50.30,92.75
7041,9993-LHIEB,Nao,Homem,0,Sim,Sim,67,Sim,Nao,DSL,...,Nao,Sim,Sim,Nao,Sim,Dois anos,Nao,Correspondencia,67.85,4627.65


In [110]:
# Verificando se a seleção foi realizada com êxito
dados_clean['Evasao'].value_counts()

Nao    5174
Sim    1869
Name: Evasao, dtype: int64

Verificamos que também há 11 valores em branco na coluna CobrancasTotais, contudo esses clientes ainda podem servir de algum valor para nossa análise. Vamos verificar o que ocorre com esta seleção específica.

In [114]:
# Seleciecionando os 11 casos especificados
dados_clean[dados_clean['CobrancasTotais'] == ' '][['Posse', 'CobrancasMensais', 'CobrancasTotais']]

,Posse,CobrancasMensais,CobrancasTotais
945,0,56.05,
1731,0,20.00,
1906,0,61.90,
2025,0,19.70,
2176,0,20.25,
2250,0,25.35,
2855,0,73.35,
3052,0,25.75,
3118,0,52.55,
4054,0,80.85,


Como observado, não são casos em que houve evasão, mas novos clientes os quais tiveram apenas uma cobrança. Isso é verificado pelo tempo de posse apresentado pelo cliente que consta como zero. Não devemos apagar estes dados, sendo assim devemos duplicar o valor do único mês cobrado para cobranças totais.

In [125]:
# Alterando os valores
sem_total = dados_clean[dados_clean['CobrancasTotais'] == ' '].index
dados_clean.loc[sem_total, 'CobrancasTotais'] = dados_clean.loc[sem_total, 'CobrancasMensais']

In [126]:
# Verificando se ainda temos valores em branco em CobrancasTotais
dados_clean[dados_clean['CobrancasTotais'] == ' '][['Posse', 'CobrancasMensais', 'CobrancasTotais']]

,Posse,CobrancasMensais,CobrancasTotais


In [127]:
# Verificando os clientes com tempo de posse igual a zero
dados_clean[dados_clean['Posse'] == 0][['Posse', 'CobrancasMensais', 'CobrancasTotais']]

,Posse,CobrancasMensais,CobrancasTotais
945,0,56.05,56.05
1731,0,20.00,20.0
1906,0,61.90,61.9
2025,0,19.70,19.7
2176,0,20.25,20.25
2250,0,25.35,25.35
2855,0,73.35,73.35
3052,0,25.75,25.75
3118,0,52.55,52.55
4054,0,80.85,80.85


In [128]:
dados_clean.to_csv('aluravoz_clean.csv', encoding = 'utf-8')